In [1]:
import os
import numpy as np
np.random.seed(123)
import pandas as pd

from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from os import listdir
from PIL import Image as PImage
import numpy as np
np.random.seed(123)
import pandas as pd
from scipy import ndimage, misc
from PIL import Image
import keras.backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, \
                         Flatten, Convolution2D, MaxPooling2D, \
                         BatchNormalization, Conv2D, Input,merge
from keras.utils import np_utils
from keras.models import Model

from skimage.io import imread
from sklearn.model_selection import train_test_split
#from spp.SpatialPyramidPooling import SpatialPyramidPooling

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, \
                         Flatten, Convolution2D, MaxPooling2D, \
                         BatchNormalization, UpSampling2D
from keras.utils import np_utils
import keras
from skimage.io import imread
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

C:\Users\Marya\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = keras.applications.resnet50.ResNet50(include_top=True, weights=None, input_tensor=None, input_shape=(512,512,1), pooling='same', classes=2)

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [12]:
model=keras.applications.vgg16.VGG16(include_top=True, weights=None, input_tensor=None, input_shape=(50,50,3), pooling=None, classes=2)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [14]:
input_shape=(50,50,3)
model = Sequential()
model.add(Conv2D(64, kernel_size=(2,2),
                 activation='relu',
                 input_shape=input_shape,padding = 'same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(128, (3, 3), activation='relu',padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(256, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(256, (3, 3), activation='relu',padding = 'same'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(512, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(512, (3, 3), activation='relu',padding = 'same'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(512, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(512, (3, 3), activation='relu',padding = 'same'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))

model.add(Dense(2, activation='softmax'))

In [15]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 50, 50, 64)        832       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 128)       73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 25, 25, 128)       147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 256)       295168    
__________

In [16]:
y=model.input
z = model.output

x = Conv2D(8, 3, activation = 'relu', padding = 'same' ,name='CV1')(z)

# 		crop1 = Cropping2D(cropping=((90,90),(90,90)))(conv1)
# 		print "crop1 shape:",crop1.shape
#
# x = Dense(100, activation="relu")(x)
# x = Dropout(0.25)(x)
# x = Dense(120, activation="relu")(x)

prediction = Dense(10, activation="softmax")(z)
model_final = Model(input = model.input, output = prediction)
# model_final.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy','mse'])
learning_rate = 0.01
decay_rate = learning_rate / 50
momentum = 0.9
sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
rm=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model_final.compile(loss='categorical_crossentropy',optimizer = 'sgd',metrics=['accuracy','mse'])

print(model_final.summary())


ValueError: Input 0 is incompatible with layer CV1: expected ndim=4, found ndim=2

In [8]:
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, \
                         Flatten, Convolution2D, MaxPooling2D, \
                         BatchNormalization, Conv2D, Input,merge,AveragePooling2D, Add
from keras.utils import np_utils
from keras.models import Model
import keras
from keras import optimizers

def relu(x):
    return Activation('relu')(x)

def neck(nip,nop,s):
    def unit(x):
        nBottleneckPlane = int(nop / 4)
        nbp = nBottleneckPlane


        x = BatchNormalization(axis=-1)(x)
        x = relu(x)
        ident = x

        x = Conv2D(nbp,(1,1),strides=(s,s))(x)

        x = BatchNormalization(axis=-1)(x)
        x = relu(x)
        x = Conv2D(nbp,3,3,border_mode='same')(x)
        x = BatchNormalization(axis=-1)(x)
        x = relu(x)
        x = Conv2D(nop,(1,1))(x)

        ident = Conv2D(nop,(1,1),strides=(s,s))(ident)

        out = Add()([ident,x])

        return out
    return unit

def cake(nip,nop,layers,std):
    def unit(x):
        for i in range(layers):
            if i==0:
                x = neck(nip,nop,std)(x)
            else:
                x = neck(nop,nop,1)(x)
        return x
    return unit

inp = Input(shape=(50,50,3))
i = inp

i = Convolution2D(16,3,3,border_mode='same')(i)

i = cake(16,32,3,1)(i) #32x32 Residual group
i = cake(32,64,3,2)(i) #16x16
i = cake(64,128,3,2)(i) #8x8

i = BatchNormalization(axis=-1)(i)
i = relu(i)

i = AveragePooling2D(pool_size=(4,4),border_mode='valid')(i) #1x1
i = Flatten()(i) # 128

i = Dense(1)(i)
i = Activation('sigmoid')(i)
lrate=0.001
model = Model(input=inp,output=i)
adam=keras.optimizers.Adam(lr=lrate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())


C:\Users\Marya\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same")`
C:\Users\Marya\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="same")`
C:\Users\Marya\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same")`
C:\Users\Marya\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same")`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 50, 50, 3)    0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 50, 50, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 50, 50, 16)   64          conv2d_11[0][0]                  
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 50, 50, 16)   0           batch_normalization_7[0][0]      
__________________________________________________________________________________________________
conv2d_12 

C:\Users\Marya\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D(pool_size=(4, 4), padding="valid")`
C:\Users\Marya\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


In [ ]:

inp = Input(shape=(320,320,3))
i = inp

i = Convolution2D(16,7,7,border_mode='same')(i)

i = cake(16,32,3,1,1)(i) #32x32
i = cake(32,64,3,2,1)(i) #16x16
i = cake(64,128,3,2,1)(i) #8x8
#i = cake(128,256,3,1,1)(i) #8x8

i = BatchNormalization(axis=-1)(i)
i = relu(i)

  
from keras import optimizers

# epochs=50
model = Model(inputs=inp, outputs=i)

ada=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer = ada,metrics=['accuracy'])
       
print(model.summary())

